1. Задача обучить Pong с помощью DQN, используя только полносвязанные слои, оставив только один канал.
2. Число эпизодов и других параметров обучения выбираются в соответствии с вычислительной мощностью.

In [ ]:
!pip install "gymnasium[atari]"
!pip install autorom[accept-rom-license]

In [ ]:
# Настройка среды Atari
import gymnasium as gym
import ale_py
gym.register_envs(ale_py)

import torch                      # библиотека для работы с нейронными сетями
import torch.nn as nn             # модуль для создания нейронных сетей
import torch.optim as optim       # модуль для оптимизации нейронных сетей
import torch.nn.functional as F   # модуль с функциями активации и потерь
import numpy as np                # библиотека для работы с массивами
from collections import deque     # структура данных для буфера воспроизведения
import random                     # модуль для генерации случайных чисел
import matplotlib.pyplot as plt   # библиотека для построения графиков
import os                         # модуль для работы с файловой системой
import base64                     # модуль для кодирования и декодирования данных
import copy                       # копирование объектов

# Поддерживает обратное распространение
from torch.autograd import Variable

# Модуль для отображения видео в Google Colab
from IPython import display as ipythondisplay

# Перевод среды в черно-белое
from gymnasium.wrappers import GrayscaleObservation
from gymnasium.wrappers import FlattenObservation

In [ ]:
env = gym.envs.make("PongDeterministic-v4", render_mode='rgb_array')

# Видео
env = gym.wrappers.RecordVideo(
    env,
    episode_trigger=lambda num: num % 100 == 0,
    video_folder="random-video-folder",
    name_prefix="video-",
)

# Переводим среду в черно-серый цвет
grayscale_env = GrayscaleObservation(env)

# Выравнивание среды
flatten_env = FlattenObservation(grayscale_env)

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/random-video-folder folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [ ]:
state_shape = flatten_env.observation_space.shape
print('Число состояний (форма):', state_shape)
n_action = flatten_env.action_space.n
print('Число действий:', n_action)

print('Доступные действия:', flatten_env.unwrapped.get_action_meanings())


Число состояний (форма): (33600,)
Число действий: 6
Доступные действия: ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [ ]:
ACTIONS = [0, 2, 3] # 'NOOP', 'RIGHT', 'LEFT'
n_action = 3

In [ ]:
class DQN():
    def __init__(self, n_state, n_action, n_hidden=50, lr=0.05):
        self.criterion = torch.nn.MSELoss()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(n_state, n_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden, n_action)
        )
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

        # Инициализируем целевую сеть
        self.model_target = copy.deepcopy(self.model)

    # Метод для синхронизации весов целевой и предсказательной сетей
    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())

    # метод обучения, который обновляет нейронную сеть, получив новый пример
    def update(self, s, y):
        """
        Обновляет веса DQN, получив обучающий пример
        @param s: состояние
        @param y: целевое значение
        """
        y_pred = self.model(torch.Tensor(s))
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    # Вычисление ценностей с помощью целевой сети
    def target_predict(self, s):
        """
        Вычисляет значения Q-функции состояния для всех действий
        с помощью целевой сети
        @param s: входное состояние
        @return: целевые ценности состояния для всех действий
        """
        with torch.no_grad():
            return self.model_target(torch.Tensor(s))

    # Для вычисления целевой ценности будем использовать целевую,
    # а  не предсказательную сеть:
    def replay(self, memory, replay_size, gamma):
        """
        Буфер воспроизведения совместно с целевой сетью
        @param memory: буфер воспроизведения опыта
        @param replay_size: сколько примеров использовать при каждом
        обновлении модели
        @param gamma: коэффициент обесценивания
        """
        if len(memory) >= replay_size:
            replay_data = random.sample(memory, replay_size)
            states = []
            td_targets = []
            for state, action, next_state, reward, is_done in replay_data:
                states.append(state)
                q_values = self.predict(state).tolist()
                if is_done:
                    q_values[action] = reward
                else:
                    q_values_next = self.target_predict(next_state).detach()
                    q_values[action] = reward + gamma * torch.max(q_values_next).item()
                td_targets.append(q_values)

            self.update(states, td_targets)

    # Функция предсказания ценности состояния:
    def predict(self, s):
        """
        Вычисляет значения Q-функции состояния для всех действий,
        применяя обученную модель
        @param s: входное состояние
        @return: значения Q для всех действий
        """
        with torch.no_grad():
            return self.model(torch.Tensor(s))



In [ ]:
def gen_epsilon_greedy_policy(estimator, epsilon, n_action):
    def policy_function(state):
        if random.random() < epsilon:
            return random.randint(0, n_action - 1)
        else:
            q_values = estimator.predict(state)
        return torch.argmax(q_values).item()
    return policy_function

In [ ]:
# n_action = 3
n_state = flatten_env.observation_space.shape[0]
n_hidden = 50
lr = 0.005

target_update = 10 # целевая сеть обновляется после каждых 10 эпизодов

n_episode = 50
replay_size = 32 # размер выборки из буфера воспроизведения на каждом шаге

# Будем запоминать полные вознаграждения в каждом эпизоде
total_reward_episode = [0] * n_episode

dqn = DQN(n_state, n_action, n_hidden, lr)

# Буфер для хранения опыта
memory = deque(maxlen=10000)

In [ ]:
def q_learning(env, estimator, n_episode, replay_size, target_update=10, gamma=1.0, epsilon=0.1, epsilon_decay=.99):
    """
    Глубокое Q-обучение методом Double DQN с воспроизведением опыта
    @param env: имя окружающей среды Gym
    @param estimator: объект класса DQN
    @param replay_size: сколько примеров использовать при каждом
    обновлении модели
    @param target_update: через сколько эпизодов обновлять целевую сеть
    @param n_episode: количество эпизодов
    @param gamma: коэффициент обесценивания
    @param epsilon: параметр ε-жад­ной стратегии
    @param epsilon_decay: коэффициент затухания epsilon
    """

    for episode in range(n_episode):
        if episode % target_update == 0:
            estimator.copy_target()

        policy = gen_epsilon_greedy_policy(estimator, epsilon, n_action)
        state, _ = env.reset()
        # state = get_state(obs)

        while True:
            action = policy(state)
            next_state, reward, done, truncated, _ = env.step(ACTIONS[action])
            total_reward_episode[episode] += reward

            # next_state = get_state(next_obs)
            memory.append((state, action, next_state, reward, done or truncated))
            if done or truncated:
                break
            estimator.replay(memory, replay_size, gamma)
            state = next_state
        print('Эпизод: {}, полное вознаграждение: {}, epsilon:{}'.
                    format(episode, total_reward_episode[episode], epsilon))
        epsilon = max(epsilon * epsilon_decay, 0.01)

In [ ]:
q_learning(flatten_env, dqn, n_episode, replay_size, target_update, gamma=.95, epsilon=.1, epsilon_decay=.99)

Эпизод: 0, полное вознаграждение: -21.0, epsilon:0.1
Эпизод: 1, полное вознаграждение: -21.0, epsilon:0.099
Эпизод: 2, полное вознаграждение: -21.0, epsilon:0.09801
Эпизод: 3, полное вознаграждение: -20.0, epsilon:0.0970299
Эпизод: 4, полное вознаграждение: -21.0, epsilon:0.096059601
Эпизод: 5, полное вознаграждение: -21.0, epsilon:0.09509900499
Эпизод: 6, полное вознаграждение: -21.0, epsilon:0.0941480149401
Эпизод: 7, полное вознаграждение: -21.0, epsilon:0.093206534790699
Эпизод: 8, полное вознаграждение: -21.0, epsilon:0.09227446944279201
Эпизод: 9, полное вознаграждение: -21.0, epsilon:0.09135172474836409
Эпизод: 10, полное вознаграждение: -21.0, epsilon:0.09043820750088044
Эпизод: 11, полное вознаграждение: -21.0, epsilon:0.08953382542587164
Эпизод: 12, полное вознаграждение: -21.0, epsilon:0.08863848717161292
Эпизод: 13, полное вознаграждение: -21.0, epsilon:0.08775210229989679
Эпизод: 14, полное вознаграждение: -21.0, epsilon:0.08687458127689782
Эпизод: 15, полное вознаграждени